## CMORPH point extraction

Run GUI, select locations, then run next code block to get CSV

In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import os
import xarray as xr

# Function to clean and convert coordinates to float
def clean_coordinate(coord):
    # Remove non-numeric characters except the dot
    cleaned = ''.join(filter(lambda x: x.isdigit() or x == '.', coord))
    return float(cleaned)

# Data as provided, sorted by Station name
data = [
    {"ID": 122, "Station": "AVA, Bah Soon Pah Road", "Lat": "1°25.04'", "Lon": "103°49.51'", "Elevation": 26},
    {"ID": 117, "Station": "Banyan Fire Station (Jurong Island)", "Lat": "1°15.36'", "Lon": "103°40.74'", "Elevation": 22},
    {"ID": 88, "Station": "Beatty Secondary School", "Lat": "1°20.50'", "Lon": "103°51.09'", "Elevation": 29},
    {"ID": 90, "Station": "CCAB Bukit Timah Road", "Lat": "1°19.15'", "Lon": "103°49.16'", "Elevation": 21},
    {"ID": 31, "Station": "CHIJ(Kellock) Primary School", "Lat": "1°16.46'", "Lon": "103°49.69'", "Elevation": 35},
    {"ID": 24, "Station": "Changi Climate Station", "Lat": "1°22.07'", "Lon": "103°58.94'", "Elevation": 15},
    {"ID": 96, "Station": "Changi Naval Base", "Lat": "1°19.05'", "Lon": "104°01.84'", "Elevation": "-"},
    {"ID": 11, "Station": "Choa Chu Kang Cemetery Office", "Lat": "1°22.46'", "Lon": "103°41.62'", "Elevation": 22},
    {"ID": 94, "Station": "Coral Secondary School", "Lat": "1°22.08'", "Lon": "103°56.94'", "Elevation": 23},
    {"ID": 103, "Station": "Dairy Farm", "Lat": "1°21.58'", "Lon": "103°46.62'", "Elevation": 64},
    {"ID": 118, "Station": "Handy Road", "Lat": "1°17.96'", "Lon": "103°50.76'", "Elevation": 21},
    {"ID": 123, "Station": "Hong Wen School", "Lat": "1°19.29'", "Lon": "103°51.46'", "Elevation": 31},
    {"ID": 63, "Station": "International Road P.S", "Lat": "1°19.65'", "Lon": "103°42.25'", "Elevation": 11},
    {"ID": 86, "Station": "JTC, Chia Ping Road", "Lat": "1°19.61'", "Lon": "103°43.23'", "Elevation": 42},
    {"ID": 101, "Station": "Jurong Junior College", "Lat": "1°21.03'", "Lon": "103°42.80'", "Elevation": 18},
    {"ID": 39, "Station": "Jurong Industrial Waterworks", "Lat": "1°19.51'", "Lon": "103°44.21'", "Elevation": 10},
    {"ID": 33, "Station": "Jurong Pier", "Lat": "1°18.49'", "Lon": "103°42.59'", "Elevation": 6},
    {"ID": 71, "Station": "Kent Ridge Road, CRISP", "Lat": "1°17.54'", "Lon": "103°46.89'", "Elevation": 72},
    {"ID": 66, "Station": "Kranji Reservoir", "Lat": "1°26.32'", "Lon": "103°44.16'", "Elevation": 9},
    {"ID": 8, "Station": "Lower Peirce Reservoir", "Lat": "1°22.20'", "Lon": "103°49.62'", "Elevation": 9},
    {"ID": 7, "Station": "Macritchie Reservoir", "Lat": "1°20.51'", "Lon": "103°50.03'", "Elevation": 25},
    {"ID": 108, "Station": "Marina Barrage", "Lat": "1°16.80'", "Lon": "103°52.22'", "Elevation": 15},
    {"ID": 119, "Station": "Marina Pumping Station", "Lat": "1°17.70'", "Lon": "103°51.73'", "Elevation": 26},
    {"ID": 111, "Station": "MEWR", "Lat": "1°18.63'", "Lon": "103°50.19'", "Elevation": 115},
    {"ID": 50, "Station": "Ngee Ann Polytechnic", "Lat": "1°19.91'", "Lon": "103°46.57'", "Elevation": 69},
    {"ID": 107, "Station": "National Sailing Centre", "Lat": "1°18.80'", "Lon": "103°57.72'", "Elevation": 9},
    {"ID": 44, "Station": "National Technological University", "Lat": "1°20.71'", "Lon": "103°41.00'", "Elevation": 84},
    {"ID": 92, "Station": "NACLI, South Buona Vista", "Lat": "1°17.05'", "Lon": "103°47.33'", "Elevation": 38},
    {"ID": 109, "Station": "Nanyang Polytechnic", "Lat": "1°22.76'", "Lon": "103°51.00'", "Elevation": 53},
    {"ID": 61, "Station": "Ping Yi Sec School", "Lat": "1°19.63'", "Lon": "103°55.24'", "Elevation": 24},
    {"ID": 114, "Station": "Bukit Panjang Govt High School", "Lat": "1°22.93'", "Lon": "103°44.29'", "Elevation": 25},
    {"ID": 64, "Station": "Bukit Panjang Telecom", "Lat": "1°22.94'", "Lon": "103°45.64'", "Elevation": 32},
    {"ID": 29, "Station": "Pasir Ris Home Team Chalets", "Lat": "1°23.19'", "Lon": "103°56.48'", "Elevation": 10},
    {"ID": 6, "Station": "Paya Lebar Meteorological Station", "Lat": "1°21.43'", "Lon": "103°54.22'", "Elevation": 16},
    {"ID": 110, "Station": "Peicai Secondary School", "Lat": "1°21.92'", "Lon": "103°52.25'", "Elevation": 57},
    {"ID": 72, "Station": "Prince Edward Road", "Lat": "1°16.43'", "Lon": "103°50.89'", "Elevation": 29},
    {"ID": 78, "Station": "Poole Road Pumping Station", "Lat": "1°18.42'", "Lon": "103°53.44'", "Elevation": 6},
    {"ID": 77, "Station": "Queenstown Secondary School", "Lat": "1°17.62'", "Lon": "103°48.76'", "Elevation": 24},
    {"ID": 81, "Station": "Punggol Secondary School", "Lat": "1°24.17'", "Lon": "103°54.57'", "Elevation": 25},
    {"ID": 105, "Station": "Northwest Regional Office, NEA", "Lat": "1°27.49'", "Lon": "103°47.72'", "Elevation": 28},
    {"ID": 102, "Station": "Semakau Island", "Lat": "1°11.41'", "Lon": "103°45.94'", "Elevation": 6},
    {"ID": 25, "Station": "Seletar Meteorological Station", "Lat": "1°25.00'", "Lon": "103°51.90'", "Elevation": 9},
    {"ID": 80, "Station": "Sembawang Meteorological Station", "Lat": "1°25.50'", "Lon": "103°49.20'", "Elevation": 26},
    {"ID": 120, "Station": "Botanic Garden", "Lat": "1°18.52'", "Lon": "103°49.08'", "Elevation": 39},
    {"ID": 84, "Station": "Simei Avenue", "Lat": "1°20.66'", "Lon": "103°56.64'", "Elevation": 19},
    {"ID": 46, "Station": "Singapore Golf Club", "Lat": "1°20.49'", "Lon": "103°48.65'", "Elevation": 34},
    {"ID": 40, "Station": "Singapore Orchids", "Lat": "1°24.40'", "Lon": "103°46.99'", "Elevation": 38},
    {"ID": 60, "Station": "Sentosa", "Lat": "1°15.02'", "Lon": "103°49.65'", "Elevation": 37},
    {"ID": 23, "Station": "Tengah Meteorological Station", "Lat": "1°23.10'", "Lon": "103°42.70'", "Elevation": 17},
    {"ID": 113, "Station": "Tao Nan School", "Lat": "1°18.30'", "Lon": "103°54.67'", "Elevation": 25},
    {"ID": 115, "Station": "Tuas Marine Transfer Station", "Lat": "1°17.30'", "Lon": "103°38.27'", "Elevation": 6},
    {"ID": 82, "Station": "Tuas Incinerator", "Lat": "1°19.49'", "Lon": "103°38.11'", "Elevation": 45},
    {"ID": 97, "Station": "Tuas Naval Base", "Lat": "1°17.63'", "Lon": "103°39.84'", "Elevation": "-"},
    {"ID": 79, "Station": "TripleOne Somerset Building", "Lat": "1°18.02'", "Lon": "103°50.23'", "Elevation": 67},
    {"ID": 35, "Station": "Ulu Pandan Sewage Treatment Work", "Lat": "1°19.95'", "Lon": "103°45.30'", "Elevation": 13},
    {"ID": 69, "Station": "Upper Peirce Reservoir", "Lat": "1°22.22'", "Lon": "103°48.28'", "Elevation": 45},
    {"ID": 43, "Station": "Upper Air Observatory", "Lat": "1°20.44'", "Lon": "103°53.29'", "Elevation": 36},
    {"ID": 106, "Station": "Pulau Ubin", "Lat": "1°25.01'", "Lon": "103°58.04'", "Elevation": 27},
    {"ID": 112, "Station": "Sarimbun (Scouts Campsite)", "Lat": "1°26.33'", "Lon": "103°42.10'", "Elevation": 6},
    {"ID": 104, "Station": "Republic Polytechnic", "Lat": "1°26.63'", "Lon": "103°47.12'", "Elevation": 27},
    {"ID": 55, "Station": "Woodbridge Hospital", "Lat": "1°23.02'", "Lon": "103°53.16'", "Elevation": 29},
    {"ID": 36, "Station": "Woodleigh Filters", "Lat": "1°20.26'", "Lon": "103°51.97'", "Elevation": 9},
    {"ID": 91, "Station": "Yishun Avenue 5", "Lat": "1°25.80'", "Lon": "103°49.84'", "Elevation": 43}
]

# Sort data alphabetically by Station
data.sort(key=lambda x: x["Station"])
selected_stations = []
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Station Selection ERA5")
        
        self.check_vars = {}
        self.checkbuttons = []

        # Number of columns for layout
        num_columns = 4
        row = 0
        col = 0

        # Create checkboxes for each station
        for station in data:
            var = tk.BooleanVar()
            chk = tk.Checkbutton(root, text=station["Station"], variable=var)
            chk.grid(row=row, column=col, sticky='w', padx=5, pady=5)
            self.check_vars[station["Station"]] = var
            self.checkbuttons.append(chk)

            # Update row and column for next checkbox
            col += 1
            if col >= num_columns:
                col = 0
                row += 1

        # Next button
        self.next_button = tk.Button(root, text="Next", command=self.get_selected_stations)
        self.next_button.grid(row=row + 1, column=0, columnspan=num_columns)

    def get_selected_stations(self):
        for station in data:
            if self.check_vars[station["Station"]].get():
                lat = round(clean_coordinate(station["Lat"])/100,3)
                lon = round(clean_coordinate(station["Lon"])/100,3)
                selected_stations.append((station["Station"], lat, lon))
        
        # Display selected stations
        messagebox.showinfo("Selected Stations", selected_stations)
        print(selected_stations)

if __name__ == "__main__":
    root = tk.Tk()
    app = App(root)
    root.mainloop()


Run this code to download all into one csv

In [ ]:
# Define the path to the directory containing the merged NetCDF files
merged_dir = r"c:\Users\userAdmin\Desktop\Climate Database\CMORPH\merged_cmorph"

# Initialize an empty DataFrame to store the results
columns = ['Datetime', 'Season'] + [station[0] for station in selected_stations]
results = pd.DataFrame(columns=columns)

# Function to determine the season
seasons = {
    'DJF': [12, 1, 2],
    'MAM': [3, 4, 5],
    'JJA': [6, 7, 8],
    'SON': [9, 10, 11]
}

def get_season(month):
    for season, months in seasons.items():
        if month in months:
            return season
    return None

def process_nc_file(file_path):
    # Open the NetCDF file with xarray
    try:
        ds = xr.open_dataset(file_path)
    except Exception as e:
        print(f"Error opening file {file_path}: {e}")
        return []

    # Extract the time series from the dataset
    time_series = pd.to_datetime(ds.time.values)
    time_series_data = []

    # Iterate through each time step in the dataset
    for time_index in range(len(time_series)):
        formatted_time = time_series[time_index].strftime('%Y-%m-%d %H:%M:%S')
        season = get_season(time_series[time_index].month)
        row = {'Datetime': formatted_time, 'Season': season}

        # Extract data for all coordinates of interest in a single pass
        for location in selected_stations:
            lat = location[1]
            lon = location[2]
            name = location[0]
            data = ds.sel(lat=lat, lon=lon, method='nearest')
            rain = data['cmorph'].isel(time=time_index).values
            row[name] = rain

        time_series_data.append(row)

    ds.close()
    return time_series_data

# Loop through all NetCDF files in the directory
for filename in os.listdir(merged_dir):
    print(f"Processing {filename}")
    if filename.endswith('.nc'):
        file_path = os.path.join(merged_dir, filename)
        time_series_data = process_nc_file(file_path)
        if time_series_data:
            results = pd.concat([results, pd.DataFrame(time_series_data)], ignore_index=True)

# Sort the combined DataFrame by Datetime
results = results.sort_values(by='Datetime').reset_index(drop=True)

# Save to a single CSV file
csv_filename = 'extracted_data_cmorph.csv'
results.to_csv(csv_filename, index=False)
print(f"Data processing complete. Results saved to '{csv_filename}'.")

# Optionally, display the first few rows of the DataFrame
print(results.head())